In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [4]:
def build_generator(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim))
    model.add(LeakyReLU(0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(Dense(output_dim, activation='tanh'))
    return model

def build_discriminator(input_dim):
    model = Sequential()
    model.add(Dense(512, input_dim=input_dim))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.3))
    model.add(Dense(128))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [6]:
class TabularGAN(Model):
    def __init__(self, generator, discriminator, latent_dim, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.generator = generator
        self.discriminator = discriminator
        self.latent_dim = latent_dim

    def compile(self, g_opt, d_opt, g_loss, d_loss, *args, **kwargs):
        super().compile(*args, **kwargs)
        self.g_opt = g_opt
        self.d_opt = d_opt
        self.g_loss = g_loss
        self.d_loss = d_loss

    def train_step(self, real_data):
        batch_size = tf.shape(real_data)[0]
        noise = tf.random.normal((batch_size, self.latent_dim))

        # Train discriminator
        with tf.GradientTape() as d_tape:
            fake_data = self.generator(noise, training=False)
            # Get discriminator predictions for real and fake data
            real_output = self.discriminator(real_data, training=True)
            fake_output = self.discriminator(fake_data, training=True)
            d_loss_real = self.d_loss(tf.ones_like(real_output), real_output)
            d_loss_fake = self.d_loss(tf.zeros_like(fake_output), fake_output)
            total_d_loss = d_loss_real + d_loss_fake

        d_grads = d_tape.gradient(total_d_loss, self.discriminator.trainable_variables)
        self.d_opt.apply_gradients(zip(d_grads, self.discriminator.trainable_variables))

        # Train generator
        with tf.GradientTape() as g_tape:
            noise = tf.random.normal((batch_size, self.latent_dim))
            fake_data = self.generator(noise, training=True)
            fake_output = self.discriminator(fake_data, training=False)
            total_g_loss = self.g_loss(tf.ones_like(fake_output), fake_output)

        g_grads = g_tape.gradient(total_g_loss, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(g_grads, self.generator.trainable_variables))

        return {"d_loss": total_d_loss, "g_loss": total_g_loss}

In [7]:
# Hyperparameters
latent_dim = 100
data_dim = 10  # Adjust this based on your tabular data dimensions

In [8]:
# Build and compile the model
generator = build_generator(latent_dim, data_dim)
discriminator = build_discriminator(data_dim)

/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
g_opt = Adam(learning_rate=0.0002, beta_1=0.5)
d_opt = Adam(learning_rate=0.0002, beta_1=0.5)
g_loss = BinaryCrossentropy()
d_loss = BinaryCrossentropy()

In [10]:
tabgan = TabularGAN(generator, discriminator, latent_dim)
tabgan.compile(g_opt, d_opt, g_loss, d_loss)